<a href="https://colab.research.google.com/github/santhoshbvsrk/CapstoneProjects/blob/main/DataAnalytics_CDF/InsaidTelecom_CDFCapstone_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def load_datasets_from_drive():

  from google.colab import drive
  drive.mount ('/content/gdrive')

  !unzip gdrive/My\ Drive/CDF_Capstone_Project/events_data.zip

In [ ]:
def read_csv_file():
  import pandas as pd 
  import numpy as np

  pd.set_option('display.float_format', lambda x: '%18f' %x)

  events_data_df=pd.read_csv("events_data.csv")

  return events_data_df

In [ ]:
def read_dataset_genderage():

  import pandas as pd 
  import numpy as np

  !pip install mysql-connector-python-rf

  import mysql.connector
  
#this is to connect to mysql database
  mydb = mysql.connector.connect(
      host="cpanel.insaid.co",
      user="student",
      password="student",
      database="Capstone1"
    )

#declaring a cursor variable of database to hold data of a specific table
  mycursor = mydb.cursor()

#mycursor variable holds the entire data from gender_age_train table
  mycursor.execute("SELECT * FROM gender_age_train")

#using fetchall function to get data from cursor and populating in another variable
  myresult = mycursor.fetchall()

#inserting table data from variable into a dataframe
  gender_age_train_df = pd.DataFrame(myresult,columns=['device_id','Gender','Age','Age_Group'])
  return gender_age_train_df

In [ ]:
def read_dataset_phonebrandmodel():

  import pandas as pd 
  import numpy as np
  import mysql.connector
  
#this is to connect to mysql database
  mydb = mysql.connector.connect(
      host="cpanel.insaid.co",
      user="student",
      password="student",
      database="Capstone1"
    )

#declaring a cursor variable of database to hold data of a specific table
  mycursor = mydb.cursor()

#mycursor variable holds the entire data from phone_brand_device_model table
  mycursor.execute("SELECT * FROM phone_brand_device_model")

#using fetchall function to get data from cursor and populating in another variable, this overwrites data from previous load
  myresult = mycursor.fetchall()

#inserting table data from variable into a dataframe
  phone_brand_device_model_df = pd.DataFrame(myresult,columns=['device_id','Brand','Model'])
  return phone_brand_device_model_df

In [ ]:
def preprocessing_checks(events_data_df,gender_age_train_df,phone_brand_device_model_df):
  print('Datasets info is as follows:')
  print("\n Events Data Set info is:",events_data_df.info())
  print('\n Gender Age Data Set info is:\n',gender_age_train_df.info())
  print('\n Mobile Brand Model Data Set info is:\n',phone_brand_device_model_df.info())
  print("Missing values across 3 datasets are:")
  print("\n Missing values in events_data_df is:\n",events_data_df.isnull().sum())
  print("\n Missing values in gender_age_train_df is:\n",gender_age_train_df.isnull().sum())
  print("\n Missing values in phone_brand_device_model_df is:\n",phone_brand_device_model_df.isnull().sum())

In [ ]:
def preprocessing(events_data_df,phone_brand_device_model_df):
  print("\n Missing Values Before Imputation are:")
  print(events_data_df.isnull().sum())

  #Preparing list of unique latitude values for the missing deviceid records.
  latlon_df = events_data_df[events_data_df.device_id.isnull()]['latitude'].unique()
  #For imputing Missing values in DeviceId, we've fetched unique latitude for the missing deviceid and found that for that latitude there is only one deviceid throughout the dataset.
  #Hence, we'll be imputing missing deviceid values by replacing the deviceid which the same latitudeid shares
  for latitude in latlon_df:
    devid = list(events_data_df[(events_data_df.latitude == latitude) & (events_data_df.device_id.notnull())].device_id)[0]
    #print('Latitude is {} for DeviceID: {}'.format(devid,latitude))
    events_data_df.loc[events_data_df.latitude == latitude, 'device_id'] = devid
  #Missing Values post imputing DeviceID
  print("\n Missing Values After DeviceID Imputation are:")
  print(events_data_df.isnull().sum())

  #For imputing the missing values in State, we'll first find all their respecitve city values
  city_list = list(events_data_df[events_data_df.state.isnull()].city.unique())
  #Will loop over the above cities list, fetch it's relevant state value where state value is not null and then populate that to all the records where city value is matching with the city value from our list
  for city in city_list:
    state = list(events_data_df[(events_data_df.city == city) & (events_data_df.state.notnull())].state)[0]
    #print("State is {} for City {}".format(state,city))
    events_data_df.loc[events_data_df.city == city, 'state'] = state
  #Missing Values post imputing State
  print("\n Missing Values After State Imputation are:")
  print(events_data_df.isnull().sum())

  #checking if the device_id is different for the records where latitude & longitude are null
  dev_id = events_data_df[(events_data_df.latitude.isnull()) & (events_data_df.longitude.isnull())].sort_values(by=['device_id'])['device_id'].unique()
  #Imputing missing values in latitude & longitude
  #From the previous steps we found that there are 21 device_ids for the missing latitude & longitude records.
  #For these 21 device_ids there are unique latitude & longitude combination. Hence, we'll be replacing the missing latitude & longitude values for these devices with the same latitude & longitude values
  for i in dev_id:
    lat = list(events_data_df[(events_data_df.device_id == i) & (events_data_df.latitude.notnull())].latitude)[0]
    lon = list(events_data_df[(events_data_df.device_id == i) & (events_data_df.latitude.notnull())].longitude)[0]
    city = list(events_data_df[events_data_df.device_id == i].city)[0]
    state = list(events_data_df[events_data_df.device_id == i].state)[0]
    #print('For DeviceID {} Latitude is: {}, Longitude is: {}, city is: {} & State is: {}'.format(i,lat,lon,city,state))
    events_data_df.loc[events_data_df.device_id == i, 'latitude'] = lat
    events_data_df.loc[events_data_df.device_id == i, 'longitude'] = lon
 #Missing Values post imputing Latitude & Longitude
  print("\n Missing Values After Latitude & Longitude Imputation are:")
  print(events_data_df.isnull().sum())

  #To replace Non-English Characters in Brand column of phone_brand_device_model dataframe
  #Add a brand name per brand in Chinese for which we don´t have a translation
  phone_brands_mapping = {"三星": "samsung","天语": "Ktouch", "海信": "hisense", "联想": "lenovo", "欧比": "obi",
                                "爱派尔": "ipair", "努比亚": "nubia", "优米": "youmi", "朵唯": "dowe", "黑米": "heymi",
                                "锤子": "hammer", "酷比魔方": "koobee", "美图": "meitu", "尼比鲁": "nibilu", "一加": "oneplus",
                                "优购": "yougo", "诺基亚": "nokia", "糖葫芦": "candy", "中国移动": "ccmc", "语信": "yuxin",
                                "基伍": "kiwu", "青橙": "greeno", "华硕": "asus", "夏新": "panasonic", "维图": "weitu",
                                "艾优尼": "aiyouni", "摩托罗拉": "moto", "乡米": "xiangmi", "米奇": "micky", "大可乐": "bigcola",
                                "沃普丰": "wpf", "神舟": "hasse", "摩乐": "mole", "飞秒": "fs", "米歌": "mige", "富可视": "fks",
                                "德赛": "desci", "梦米": "mengmi", "乐视": "lshi", "小杨树": "smallt", "纽曼": "newman",
                                "邦华": "banghua", "E派": "epai", "易派": "epai", "普耐尔": "pner", "欧新": "ouxin", "西米": "ximi",
                                "海尔": "haier", "波导": "bodao", "糯米": "nuomi", "唯米": "weimi", "酷珀": "kupo", "谷歌": "google",
                                "昂达": "ada", "聆韵": "lingyun", "小米": "Xiaomi", "华为": "Huawei", "魅族": "Meizu", "中兴": "ZTE",
                                "酷派": "Coolpad", "金立": "Gionee", "SUGAR": "SUGAR", "OPPO": "OPPO", "vivo": "vivo", "HTC": "HTC",
                                "LG": "LG", "ZUK": "ZUK", "TCL": "TCL", "LOGO": "LOGO", "SUGAR": "SUGAR", "Lovme": "Lovme",
                                "PPTV": "PPTV", "ZOYE": "ZOYE", "MIL": "MIL", "索尼" : "Sony", "欧博信" : "Opssom", "奇酷" : "Qiku",
                                "酷比" : "CUBE", "康佳" : "Konka", "亿通" : "Yitong", "金星数码" : "JXD", "至尊宝" : "Monkey King",
                                "百立丰" : "Hundred Li Feng", "贝尔丰" : "Bifer", "百加" : "Bacardi", "诺亚信" : "Noain", 
                                "广信" : "Kingsun", "世纪天元" : "Ctyon", "青葱" : "Cong", "果米" : "Taobao", "斐讯" : "Phicomm",
                                "长虹" : "Changhong", "欧奇" : "Oukimobile", "先锋" : "XFPLAY", "台电" : "Teclast", "大Q" : "Daq",
                                "蓝魔" : "Ramos", "奥克斯" : "AUX", "索尼" : "Sony", "欧博信" : "Opssom", "奇酷" : "Qiku",
                                "酷比" : "CUBE", "康佳" : "Konka", "亿通" : "Yitong", "金星数码" : "JXD", "至尊宝" : "Monkey King",
                                "百立丰" : "Hundred Li Feng", "贝尔丰" : "Bifer", "百加" : "Bacardi", "诺亚信" : "Noain",
                                "广信" : "Kingsun", "世纪天元" : "Ctyon", "青葱" : "Cong", "果米" : "Taobao", "斐讯" : "Phicomm",
                                "长虹" : "Changhong", "欧奇" : "Oukimobile", "先锋" : "XFPLAY", "台电" : "Teclast", "大Q" : "Daq", 
                                "蓝魔" : "Ramos", "奥克斯" : "AUX", "飞利浦": "Philips", "智镁": "Zhimei", "惠普": "HP",
                                "原点": "Origin", "戴尔": "Dell", "碟米": "Diemi", "西门子": "Siemens", "亚马逊": "Amazon",
                                "宏碁": "Acer",
                                '世纪星': "UB1", '丰米': "UB2", '优语':'UB3', '凯利通': "UB4", '唯比': "UB5", '嘉源': "UB6",
                                 '大显': "UB7", '天宏时代': "UB8", '宝捷讯': 'UB9','帷幄': 'UB10', '德卡诺': 'UB11',
                                '恒宇丰': 'UB12', '本为': 'UB13', '极米': 'UB14', '欧乐迪': 'UB15', '欧乐酷': 'UB16',
                                '欧沃': 'UB17', '瑞米': 'UB18', '瑞高': 'UB19', '白米': 'UB20', '虾米': 'UB21', '赛博宇华': 'UB22',
                                '首云': 'UB23', '鲜米': 'UB24','E人E本':'eben'}

  phone_brand_device_model_df.Brand = phone_brand_device_model_df.Brand.apply(lambda x: phone_brands_mapping.setdefault(x.strip(),x.strip()))
  print("\n After mapping Non-English Characters in Brand: \n",phone_brand_device_model_df.Brand.unique())
  

In [ ]:
def filter_data(events_data_df):
  #creating a dataframe which contains event data for only 6 states under consideration
  events_data_filtered_df= events_data_df[(events_data_df.state == 'WestBengal') | (events_data_df.state == 'Karnataka') | (events_data_df.state == 'Gujarat') | (events_data_df.state == 'Bihar') | (events_data_df.state == 'Punjab') | (events_data_df.state == 'Kerala')]
  return events_data_filtered_df

In [ ]:
def validate_latlong(events_data_filtered_df,events_data_df):
  !pip install chart_studio

  # Making plotly specific imports
  # These imports are necessary to use plotly offline without signing in to their website.
  from plotly.offline import init_notebook_mode, iplot
  import plotly.graph_objects as go
  import chart_studio.plotly as py
  from plotly import tools
  init_notebook_mode(connected=True)
  
  import plotly.express as px
  fig = px.scatter_mapbox(events_data_filtered_df, lat="latitude", lon="longitude", zoom=3,hover_name='city')
  fig.update_layout(mapbox_style="open-street-map")  # <== Using Mapbox
  #fig.show()

  #checking the number of unique devices for that latitude & longitude
  events_data_df[(events_data_df.latitude == 34.555300) & (events_data_df.longitude == 69.207500)]['device_id'].nunique()
  events_data_df[(events_data_df.latitude == 25.204800) & (events_data_df.longitude == 55.270800)]['device_id'].nunique()
  events_data_df[(events_data_df.latitude == 41.871900) & (events_data_df.longitude == 12.567400)]['device_id'].nunique()

  #checking if the device_id is different for the records where latitude & longitude are pointing outside India
  dev_id = events_data_df[(((events_data_df.latitude.isnull()) & (events_data_df.longitude.isnull())) | ((events_data_df.latitude == 34.555300) & (events_data_df.longitude == 69.207500)) | ((events_data_df.latitude == 25.204800) & (events_data_df.longitude == 55.270800)) | ((events_data_df.latitude == 41.871900) & (events_data_df.longitude == 12.567400)))].sort_values(by=['device_id'])['device_id'].unique()

  #Imputing wrong values in latitude & longitude
  #From the previous steps we found that there are 20 device_ids for the wrong values in latitude & longitude records.
  #For these 20 device_ids there are unique latitude & longitude combination. Hence, we'll be replacing the missing latitude & longitude values for these devices with the same latitude & longitude values

  for i in dev_id:
    lat = list(events_data_df[(events_data_df.device_id == i) & (events_data_df.latitude.notnull())].latitude)[0]
    lon = list(events_data_df[(events_data_df.device_id == i) & (events_data_df.latitude.notnull())].longitude)[0]
    city = list(events_data_df[events_data_df.device_id == i].city)[0]
    state = list(events_data_df[events_data_df.device_id == i].state)[0]
    #print('For DeviceID {} Latitude is: {}, Longitude is: {}, city is: {} & State is: {}'.format(i,lat,lon,city,state))
    events_data_df.loc[events_data_df.device_id == i, 'latitude'] = lat
    events_data_df.loc[events_data_df.device_id == i, 'longitude'] = lon

  print('\n Devices with latitude & longitude which are outside India post Imputation are: ',events_data_df[(((events_data_df.latitude == 34.555300) & (events_data_df.longitude == 69.207500)) | ((events_data_df.latitude == 25.204800) & (events_data_df.longitude == 55.270800)) | ((events_data_df.latitude == 41.871900) & (events_data_df.longitude == 12.567400)))]['device_id'].count())

In [ ]:
def noterun():
  #function to load events_data.csv into workspace
  load_datasets_from_drive()
  #function to read event_data into dataframe, gender_age & mobile_brand tables data into dataframes, get info of all 3 dataframes & check for missing values counts in all 3 dataframes
  preprocessing_checks(read_csv_file(), read_dataset_genderage(), read_dataset_phonebrandmodel()) 
  #function to perform preprocessing on all 3 dataframes wherever applicable to handle missing values
  preprocessing(read_csv_file(), read_dataset_phonebrandmodel()) 
  #function to validate latitude & longitude on dataset which is filtered for 6 states
  validate_latlong(filter_data(read_csv_file()),read_csv_file())

In [ ]:
noterun()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Archive:  gdrive/My Drive/CDF_Capstone_Project/events_data.zip
replace events_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Datasets info is as follows:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3252950 entries, 0 to 3252949
Data columns (total 7 columns):
 #   Column     Dtype  
---  ------     -----  
 0   event_id   int64  
 1   device_id  float64
 2   timestamp  object 
 3   longitude  float64
 4   latitude   float64
 5   city       object 
 6   state      object 
dtypes: float64(3), int64(1), object(3)
memory usage: 173.7+ MB

 Events Data Set info is: None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74645 entries, 0 to 74644
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   device_id  74645 non-null  int64 
 1   Gender     74645 non-null  object
 2   Age        74645 no


 Devices having the latitude & longitude which are outside India post Imputation are:  0
